In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import numpy as np
import pickle
import re

In [22]:
#log in to LinkedIn to access data
def login(driver):
    url = "https://www.linkedin.com/checkpoint/rm/sign-in-another-account?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin"
    wait = WebDriverWait(driver, 10)
    driver.get(url)

    username = driver.find_element_by_id("username")
    password = driver.find_element_by_id("password")
    username.send_keys("????")    #USERNAME to LI
    password.send_keys("????")    #PASSWORD to LI

def search(driver):
    time.sleep(20)
    driver.get("https://www.linkedin.com/jobs/search/?keywords=data%20scientist&location=New%20York%2C%20United%20States")

#grabs results fetched
def get_n_results(driver):
  time.sleep(2)
  results_div = driver.find_element_by_xpath('//*[@id="main"]/div/div[2]/div[1]/header/div[1]/small/div/span')
  n_string = results_div.text
  n = int(n_string.split()[0].replace(',',""))
  return n 

#Finds job ul div
def get_jobs(driver):
  ul_div = driver.find_element_by_xpath("//main[@id='main']//ul[1]")
  return ul_div
    
#Scrolls to properly load page
def scroll_down(driver):
    time.sleep(2)
    element = driver.find_element_by_xpath('//*[@id="main"]/div/div[2]/div[1]/div')
    scroll_height = element.get_property("scrollHeight")

    # Set the scroll step and delay between steps
    scroll_step = 150
    delay = 0.1

    current_scroll = 0
    while current_scroll < scroll_height:
        driver.execute_script("arguments[0].scrollTo(0, arguments[1]);", element, current_scroll)
        time.sleep(delay)
        current_scroll += scroll_step

    # Wait for a short period for the new content to load
    time.sleep(2)
      
def get_job_urls(jobs,driver,job_urls = {}):
  i = 1
  #Collects job urls,location role cand company 
  #the final result updates the input dictionary and appends a key value pair with the format
  #    url:{'company':company,'location':location,'role':role}
  while True: 
    try:
        WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH,f"//main[@id='main']//ul[1]")))
        url = jobs.find_element_by_xpath(f"//main[@id='main']//ul[1]//li[{i}]//a[1]").get_attribute("href")
        role = jobs.find_element_by_xpath(f"//main[@id='main']//ul[1]//li[{i}]//a[1]").text
        company = jobs.find_element_by_xpath(f"//main[@id='main']//ul[1]//li[{i}]//div[1]//div[1]//div[1]//div[2]//div[2]//span[1]").text
        location = driver.find_element_by_xpath(f"//main[@id='main']//ul[1]//li[{i}]//li[1]").text
        job_urls.update({url:{'company':company,'location':location,'role':role}})
        i+=1
    except:
        return job_urls

def load_next_page(driver):
  #loads next page for url retrival
  curr= driver.find_element_by_xpath('//*[@aria-current="true"]').text
  next = driver.find_element_by_xpath(f'//*[@aria-label="Page {int(curr)+1}"]')
  next.click()

In [ ]:
# Run theough the entire process of fetch in urls logining in and grabing job descriptions
driver = webdriver.Chrome(r'C:\Program Files\Google\Chrome\Application\chromedriver.exe')
login(driver)
search(driver)
n = get_n_results(driver)
if n > 350: #makes sure only 350 or less job openings are scraped (too much time otherwise)
    n = 350
pages = int(n/25)
job_dict ={}
#iterate through the amount of pages given
for i in range(pages):
  scroll_down(driver)
  jobs = get_jobs(driver)
  get_job_urls(jobs,driver,job_urls = job_dict)
  load_next_page(driver)





In [24]:
def get_description(driver, job_dict):
    fail = []
    # Iterate through the url list to scrape the descriptions
    for url in list(job_dict.keys()):
        try:
            driver.get(url)
            time.sleep(2)
            if driver.current_url != url:
                print(f'failed at {url}')
                # remove broken urls
                job_dict.pop(url)
            # scrape
            applicants_str = driver.find_element_by_xpath("//*[self::span or self::strong][contains(text(), 'applicant') or contains(text(), 'applicants')]").text
            applicant_num = int(''.join(filter(str.isdigit, applicants_str)))
            
            post_dates = driver.find_elements_by_xpath("//*[contains(text(), 'ago')]")
            for post_date in post_dates:
                text = post_date.text
                post_date_days = re.search(r'(\d+)\s+(month|week|day|hour|minute|second)s?\s+ago', text)
                if post_date_days:
                    value = int(post_date_days.group(1))
                    unit = post_date_days.group(2)
            
                    if unit == "month":
                        value *= 30
                    elif unit == "week":
                        value *= 7
                    elif unit in ["day", "hour", "minute", "second"]:
                        # Handle other units as needed
                        pass
                
            description_element = driver.find_element_by_xpath("//h2[text()='About the job']/parent::div")
            description = description_element.get_attribute("textContent")
            job_dict.get(url).update({"applicants": applicant_num, "description": description, "post_days": value})
        except Exception as e:
            print(f"An error occurred while processing {url}: {str(e)}")
            fail.append(url)
    return job_dict

In [ ]:
job_dict = get_description(driver,job_dict)

In [26]:
# save dict to CSV
import csv
import datetime

# Get the current date and time
current_datetime = datetime.datetime.now()

# Format the date and time as a string
datetime_string = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")

# Append the formatted date and time to the original filename
filename = f"job_data_scientist_NYC_{datetime_string}.csv"# Specify the file path and filename 

with open(filename, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Write the header row
    writer.writerow(['url', 'company', 'location', 'role', 'description', 'applicants','post_days'])
    
    # Write the data row
    for url, data in job_dict.items():
        row = [url] + [data.get(key, '') for key in ['company', 'location', 'role', 'description', 'applicants','post_days']]
        writer.writerow(row)